# Install requirements
Make sure you're using the local conda env for running this notebook. If is not created yet, create one with python 3.9 by running `conda create --name myenv python=3.9`

In [1]:
! pip install --upgrade pip

In [2]:
! pip install -r "../requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 19.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 6.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 6.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━

# Load dataset

In [3]:
from datasets import Dataset, load_dataset, load_from_disk, concatenate_datasets
dataset_name = "code_search_net"

def load_from_cs_net(take: int) -> Dataset:
  ds = load_dataset(dataset_name, 'python', split='train')
  return Dataset.from_dict(ds[:take]) # type: ignore

/Users/beto/Projects/embedding_comparator/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Embedding models

In [4]:
from sentence_transformers import SentenceTransformer

comment_model = SentenceTransformer('all-mpnet-base-v2')
code_model = SentenceTransformer('flax-sentence-embeddings/st-codesearch-distilroberta-base')
embedding_shape = (768)

# Generate negative samples

In [5]:
from typing import Iterator
from numpy.random import default_rng


random_generator = default_rng(seed=42)

def generate_negative_samples(iterator: Iterator, negative_samples_per_sample: int):
  for batched_sample in iterator:
    codes_embeddings = batched_sample['code_embedding']
    comments_embeddings = batched_sample['comment_embedding']
    batch_indexes = range(len(codes_embeddings))

    for index in batch_indexes:
      indexes = [i for i in batch_indexes if i != index]
      negative_indexes = random_generator.choice(indexes, negative_samples_per_sample, replace=False)

      yield {
        "code_embedding": codes_embeddings[index],
        "comment_embedding": comments_embeddings[index],
        "target": 1
      }

      for negative_index in negative_indexes:
        yield {
          "code_embedding": codes_embeddings[index],
          "comment_embedding": comments_embeddings[negative_index],
          "target": 0
        }

def with_neg_samples(dataset: Dataset, negative_samples_per_sample: int, batch_size = 100) -> Dataset:
  assert negative_samples_per_sample <= batch_size, "negative_samples_per_sample must not be greater than batch_size"
  if negative_samples_per_sample <= 0:
    return dataset
  
  dataset_with_negative_samples: Dataset = Dataset.from_generator(lambda: generate_negative_samples(dataset.iter(batch_size=batch_size), negative_samples_per_sample)) # type: ignore
  return dataset_with_negative_samples

# Generate embedding dataset for training

In [6]:
import os


train_count = 2000
train_dataset_path = f'../datasets/embeddings_python_train_{train_count}'
train_pairs = load_from_cs_net(train_count)
is_embeddings_dataset_stored = os.path.isdir(train_dataset_path)

def generate_embeddings_in_batch(batched_sample):
  codes = batched_sample['func_code_string']
  comments = batched_sample['func_documentation_string']

  return {
    "code_embedding": code_model.encode(codes),
    "comment_embedding": comment_model.encode(comments),
  }

embeddings_dataset: Dataset = Dataset.from_dict(load_from_disk(train_dataset_path)[:train_count]) if is_embeddings_dataset_stored else train_pairs.map(
  generate_embeddings_in_batch, 
  batched=True, 
  batch_size=100,
  remove_columns=list(train_pairs[0].keys()),
  desc="Generating embeddings"
) # type: ignore

if is_embeddings_dataset_stored == False:
  embeddings_dataset.save_to_disk(train_dataset_path)

Generating validation split: 100%|██████████| 23107/23107 [00:03<00:00, 6553.27 examples/s]


# Train

In [7]:
epoch = 100
batch_size = 200

## Add negative samples to train dataset

In [8]:
def to_tf_dataset(negative_samples_per_sample: int):
  tf_train_dataset = with_neg_samples(embeddings_dataset.shuffle(), negative_samples_per_sample).to_tf_dataset().map(lambda sample: ({
    "code_embedding": sample["code_embedding"],
    "comment_embedding": sample["comment_embedding"],
  }, sample["target"]))
  
  return tf_train_dataset

## Fit

In [9]:
from keras import callbacks
from models import build_dense_model

neg_samples_count = [1, 5, 15]
num_hidden_layers = 4
for neg_count in neg_samples_count:
  model = build_dense_model(num_hidden_layers=num_hidden_layers, input_shape=embedding_shape, model_name=f'dense_{num_hidden_layers}_neg_{neg_count}')
  tf_train_dataset = to_tf_dataset(neg_count)
  tensor_board_callback = callbacks.TensorBoard(log_dir=f'../logs/{model.name}')

  model.fit(
    tf_train_dataset.batch(batch_size),
    batch_size=batch_size,
    epochs=epoch,
    callbacks=[tensor_board_callback]
  )
  model.save(f'../models/{model.name}')

2023-10-20 15:40:25.682503: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-10-20 15:40:25.682521: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-20 15:40:25.682526: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-20 15:40:25.682575: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 15:40:25.682788: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Generating train split: 4000 examples [00:00, 6523.18 examples/s]


Epoch 1/100


2023-10-20 15:40:28.218464: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2023-10-20 15:40:28.261700: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


20/20 [==============================] - 11s 280ms/step - loss: 0.6958 - binary_accuracy: 0.5002 - precision: 0.5003 - recall: 0.4930
Epoch 2/100
20/20 [==============================] - 5s 273ms/step - loss: 0.6922 - binary_accuracy: 0.5232 - precision: 0.5222 - recall: 0.5480
Epoch 3/100
20/20 [==============================] - 5s 272ms/step - loss: 0.6776 - binary_accuracy: 0.5688 - precision: 0.5711 - recall: 0.5520
Epoch 4/100
20/20 [==============================] - 5s 273ms/step - loss: 0.5770 - binary_accuracy: 0.7038 - precision: 0.6815 - recall: 0.7650
Epoch 5/100
20/20 [==============================] - 5s 274ms/step - loss: 0.4503 - binary_accuracy: 0.7885 - precision: 0.7553 - recall: 0.8535
Epoch 6/100
20/20 [==============================] - 5s 273ms/step - loss: 0.3886 - binary_accuracy: 0.8280 - precision: 0.8026 - recall: 0.8700
Epoch 7/100
20/20 [==============================] - 5s 274ms/step - loss: 0.3495 - binary_accuracy: 0.8497 - precision: 0.8207 - recall: 0.8

INFO:tensorflow:Assets written to: ../models/dense_4_neg_1/assets
Generating train split: 12000 examples [00:00, 15392.73 examples/s]

...

INFO:tensorflow:Assets written to: ../models/dense_4_neg_5/assets
Generating train split: 32000 examples [00:01, 24497.12 examples/s]


Epoch 1/100


2023-10-20 16:54:14.636278: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


160/160 [==============================] - 52s 304ms/step - loss: 0.2373 - binary_accuracy: 0.9353 - precision: 0.1354 - recall: 0.0065
Epoch 2/100
160/160 [==============================] - 51s 316ms/step - loss: 0.1899 - binary_accuracy: 0.9374 - precision: 0.4681 - recall: 0.0110
Epoch 3/100
160/160 [==============================] - 51s 318ms/step - loss: 0.1618 - binary_accuracy: 0.9385 - precision: 0.5943 - recall: 0.0520
Epoch 4/100
160/160 [==============================] - 51s 316ms/step - loss: 0.1330 - binary_accuracy: 0.9430 - precision: 0.6212 - recall: 0.2255
Epoch 5/100
160/160 [==============================] - 52s 323ms/step - loss: 0.1166 - binary_accuracy: 0.9489 - precision: 0.6533 - recall: 0.3900
Epoch 6/100
160/160 [==============================] - 51s 317ms/step - loss: 0.1036 - binary_accuracy: 0.9535 - precision: 0.6737 - recall: 0.4965
Epoch 7/100
160/160 [==============================] - 50s 313ms/step - loss: 0.0908 - binary_accuracy: 0.9612 - precision: 

INFO:tensorflow:Assets written to: ../models/dense_4_neg_15/assets


# Validation

## Generate pairs lookup dictionary

In [ ]:
from tqdm import tqdm


python_splits = load_dataset(dataset_name, 'python', split=['train', 'test', 'validation']) # type: ignore
python_full_dataset = concatenate_datasets(python_splits)
splits_info = python_splits[0].info.splits
python_full_dataset_count = sum([splits_info[key].num_examples for key in splits_info.keys()])

full_dataset_url_index = { sample['func_code_url']: index  for index, sample in tqdm(enumerate(python_full_dataset), desc="Generating dict lookup", total=python_full_dataset_count) }
def search_by_url(url: str) -> int | None:
  try:
    return full_dataset_url_index[url]
  except:
    return None

## 1. CodeSearchNet queries

In [ ]:
query_samples_path = '../datasets/query_samples'

def remove_duplicates(dataset: Dataset) -> Dataset:
  pandas_dataset = dataset.to_pandas().drop_duplicates(subset=['Language', 'Query', 'GitHubUrl', 'Relevance'], ignore_index=True) # type: ignore
  dedup_dataset = Dataset.from_pandas(pandas_dataset)
  return dedup_dataset

def remove_queries_without_code(dataset: Dataset) -> Dataset:
  return dataset.filter(lambda sample: search_by_url(sample['GitHubUrl']) is not None, desc="Filtering queries with no corresponding code")

def pre_process_query_samples() -> Dataset:
  cs_net_queries_dataset: Dataset = Dataset.from_csv('../datasets/code_search_net_queries.csv') # type: ignore
  
  return remove_queries_without_code(remove_duplicates(cs_net_queries_dataset))

def get_query_samples() -> Dataset:
  try:
    return Dataset.load_from_disk(query_samples_path)
  except:
    query_samples = pre_process_query_samples()
    query_samples.save_to_disk(query_samples_path)
    return query_samples

In [ ]:
query_samples: Dataset = get_query_samples()

### Predict

In [ ]:
def get_query_code_embeddings(samples) -> Dataset:
  query_texts = [sample['Query'] for sample in samples]
  query_codes = [python_full_dataset[search_by_url(sample['GitHubUrl'])]['func_code_string'] for sample in samples]
  assert len(query_texts) == len(query_codes), "query_texts and query_codes arrays doesn't have the same length"

  query_embeddings = comment_model.encode(query_texts)
  code_embeddings = code_model.encode(query_codes)

  validation_dataset = []
  for query_embedding, code_embedding in zip(query_embeddings, code_embeddings):
    validation_dataset.append({
      "code_embedding": code_embedding,
      "comment_embedding": query_embedding,
    })

  return Dataset.from_list(validation_dataset)

In [ ]:
from keras.models import load_model

def validate(model, samples):
  validation_dataset = get_query_code_embeddings(samples).to_tf_dataset(batch_size=10)

  return {
    "predictions": model.predict(validation_dataset, verbose=0).flatten(),
    "targets": [sample['Relevance'] for sample in samples]
  }


In [ ]:
def is_prediction_correct(prediction, target) -> bool:
  if target in [0, 1]:
    return prediction <= 0.5
  
  if target in [2, 3]:
    return prediction > 0.5
  
  raise ValueError(f"target should be in range of [0, 3]. Instead, it has value of {target}")

In [ ]:
validation_query_samples = [sample for sample in query_samples if sample['Language'].lower() == 'python']
validation_query_samples_count = len(validation_query_samples)

for model_name in os.listdir('../models/'):
  model = load_model(f'../models/{model_name}')
  result = validate(model, validation_query_samples)
  
  hits = sum([is_prediction_correct(prediction, target) for prediction, target in zip(result['predictions'], result['targets'])])
  success_percentage = hits / validation_query_samples_count

  print(f"model {model_name}: {success_percentage:.2%} - {hits} of {validation_query_samples_count}")